In [1]:
from pymongo import MongoClient
import pandas as pd
import folium

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll=connectCollection('companies','offices')

In [4]:
offices=list(coll.find())

In [5]:
offices[0].keys()

dict_keys(['_id', 'loc', 'Category'])

In [6]:
for e in offices:
    del e['_id']
    del e['Category']

In [7]:
available_offices=[]

for e in offices:
    x=e['loc']['coordinates']
    available_offices.append(e['loc']['coordinates'])
available_offices

[[40.756054, -73.986951],
 [40.7322535, -73.9874105],
 [40.753918, -73.981237],
 [40.7038257, -74.0083282],
 [40.762673, -73.969296]]

In [8]:
companies=[]

for e in range(1,6):
    companies.append(f"Company {e}")

locations=pd.DataFrame(companies)

locations=locations.rename(columns={0:'Companies'})

In [9]:
loc_dict=dict()
for i in range(len(companies)):
    loc_dict[companies[i]]=available_offices[i]

print(loc_dict)

{'Company 1': [40.756054, -73.986951], 'Company 2': [40.7322535, -73.9874105], 'Company 3': [40.753918, -73.981237], 'Company 4': [40.7038257, -74.0083282], 'Company 5': [40.762673, -73.969296]}


In [10]:
columns=['Clubs','Starbucks','Schools']

for e in columns:
    locations[e]=None

locations

,Companies,Clubs,Starbucks,Schools
0,Company 1,None,None,None
1,Company 2,None,None,None
2,Company 3,None,None,None
3,Company 4,None,None,None
4,Company 5,None,None,None


In [11]:
db, clubs=connectCollection('companies','clubs')
db, starbucks=connectCollection('companies','starbucks')
db, schools= connectCollection('companies','schools')

In [12]:
def geoquery(coordinates, collection,max_distance):
    results = db[collection].find({
        "loc": {
            "$near":{
                "$geometry": {
                    "type":"Point",
                    "coordinates":[coordinates[0],coordinates[1]]
                }, 
                "$maxDistance":max_distance
            }
        }
    })
    return len(list(results))

In [13]:
def fill_locations(collection,max_distance,column):
    close_things=[]
    for i in range(len(available_offices)):
        close_things.append(geoquery(available_offices[i],collection,max_distance))
    locations[column]=close_things
    return locations

In [14]:
fill_locations('clubs',1000,'Clubs')

fill_locations('schools',1000,'Schools')

fill_locations('starbucks',1000,'Starbucks')

,Companies,Clubs,Starbucks,Schools
0,Company 1,5,8,3
1,Company 2,4,8,3
2,Company 3,6,7,1
3,Company 4,1,5,1
4,Company 5,3,1,4


In [15]:
locations['Rank']=0

locations

,Companies,Clubs,Starbucks,Schools,Rank
0,Company 1,5,8,3,0
1,Company 2,4,8,3,0
2,Company 3,6,7,1,0
3,Company 4,1,5,1,0
4,Company 5,3,1,4,0


In [16]:
ranks=[]

for i in range(len(locations)):
    x=locations.loc[i]['Clubs']*0.25+locations.loc[i]['Schools']*0.25+locations.loc[i]['Schools']*0.25
    ranks.append(x)
locations['Rank']=ranks

In [17]:
locations

,Companies,Clubs,Starbucks,Schools,Rank
0,Company 1,5,8,3,2.75
1,Company 2,4,8,3,2.50
2,Company 3,6,7,1,2.00
3,Company 4,1,5,1,0.75
4,Company 5,3,1,4,2.75


In [18]:
map_city = folium.Map(location=(40.7142700, -74.0059700), zoom_start=12)

In [19]:
final_lst=['Company 1','Company 5']

for e in final_lst:
    folium.Marker(loc_dict[e], radius=2,
                    icon=folium.Icon(prefix='fa',icon='cloud',color='red'), 
                   ).add_to(map_city)
    
map_city